In [ ]:
import zipfile

!wget https://www.cse.iitk.ac.in/users/purushot/courses/ml/2022-23-s/material/assn/assn2.zip

zip_ref = zipfile.ZipFile("assn2.zip" ,  "r")
zip_ref.extractall()
zip_ref.close()

--2023-08-21 14:00:49--  https://www.cse.iitk.ac.in/users/purushot/courses/ml/2022-23-s/material/assn/assn2.zip
Resolving www.cse.iitk.ac.in (www.cse.iitk.ac.in)... 202.3.77.10
Connecting to www.cse.iitk.ac.in (www.cse.iitk.ac.in)|202.3.77.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59842602 (57M) [application/zip]
Saving to: ‘assn2.zip.1’

assn2.zip.1         100%[===================>]  57.07M   966KB/s    in 27s     

2023-08-21 14:01:17 (2.12 MB/s) - ‘assn2.zip.1’ saved [59842602/59842602]



In [ ]:
import os

num= len(os.listdir("assn2/train/"))
num

2001

### Loading data and preprossing

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image as img
# Directory containing the images
def load_images(directory):
  #directory = 'assn2/train'
  num= len(os.listdir(directory))-1 ;
  print(num)
  # List to store the loaded images
  images = []

  # Iterate over the files in the directory
  for i in range(num):
    filename = f"{i}.png"
    # Construct the full file path
    #print(filename)
    file_path = os.path.join(directory, filename)

    # Load the image using OpenCV
    image = img.open(file_path)
    """"""
    test_image_array = np.array(image)
    red,green,blue,alpha = image.split()
    #test_image.show()
    alpha_3 = np.array(alpha)
    #alpha_3 = np.where(alpha_3==255,0,alpha_3)
    #cv2_imshow(alpha_3)
    #alpha_3 = np.where(alpha_3>0,255,alpha_3)
    #cv2_imshow(alpha_3)
    image_bgr = cv2.imread(os.path.join(directory, filename))
    # Convert BGR image to HSV
    image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
    # Access individual channels (H, S, V)
    h, s, v = cv2.split(image_hsv)
    hue_channel = image_hsv[:,:,0]
    index = np.where(hue_channel==np.max(hue_channel))
    index_s = np.where(s<250)
    index_v = np.where(v<253)
    index_h = np.where(h>100)
    #index_2 = np.where(alpha_3<255)
    #alpha_3 = np.where(hue_channel==hue_channel[0][0],alplha_3=255,alpha_3)
    index_2 = np.where(test_image_array[:,:,3]<255)
    test_image_array[index_2] = [255,255,255,0]
    #cv2_imshow(test_image_array)
    test_image_array[index_s] = [255,255,255,0]
    #cv2_imshow(test_image_array)
    right_half = test_image_array[:, 368:446]
    #cv2_imshow(right_half)
    ###########shi################
    final_hsv = cv2.cvtColor(right_half, cv2.COLOR_BGR2HSV)
    np.unique(final_hsv[:,:,0])
    final_h, final_s, final_v = cv2.split(final_hsv)
    '''t = np.unique(final_h, return_counts=True)
    print("t:",t)
    ind = np.where(t[1]==np.max(t[1][1:]))
    #print(t[ind[0][0]])
    last_index = np.where(final_h!=t[0][ind])
    right_half[last_index] = [255]
    cv2_imshow(right_half)
    cv2_imshow(cv2.cvtColor(right_half,cv2.COLOR_BGR2GRAY))'''
    t = np.unique(final_hsv, return_counts=True)
    #print("t:",t)
    ind = np.where(t[1]==np.max(t[1][1:]))
    #print(t[ind[0][0]])
    last_index = np.where(final_hsv>200)
    right_half[last_index] = [255]
    #cv2_imshow(right_half)
    #cv2_imshow(cv2.cvtColor(right_half,cv2.COLOR_BGR2GRAY))
    # Add the image to the list
    images.append(cv2.resize(cv2.cvtColor(right_half,cv2.COLOR_BGR2GRAY),(28,28)))
  '''for i in range(num):
    images[i]=images[i][:,350:450]'''

  images= np.array(images)
  return images


###Loading labels from the folder and making them into format of 0, 1

In [ ]:
def load_labels(directory):
  def convert_labels(labels):
    mapping = {'ODD': 0, 'EVEN': 1}
    converted_labels = [mapping[label] for label in labels]
    return converted_labels

  filename="labels.txt"
  file_path = os.path.join(directory, filename)
  # reading the file
  my_file = open(file_path , "r")
  data = my_file.read()

  # replacing end splitting the text
  # when newline ('\n') is seen.
  data_into_list = data.split("\n")
  data_len= len(data_into_list)-1
  data_into_list=data_into_list[:data_len]
  print(len(data_into_list))
  my_file.close()
  return np.array(convert_labels(data_into_list))

In [ ]:
images = load_images("assn2/train")
labels = load_labels("assn2/train")

2000
2000


In [ ]:
images.shape

(2000, 28, 28)

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(images[2])

In [ ]:
labels[2]

0

In [ ]:
labels.shape

(2000,)

##Standardization

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten

In [ ]:
def img_flatten(X):
  X_train =[]
  num = len(X)
  print(num)
  X = X/255.
  for i in range(num):
    X_train.append( X[i].flatten())

  X_train = np.array(X_train)
  return X_train

In [ ]:
X_train = img_flatten(images)

In [ ]:
X_train.shape , labels.shape

((2000, 784), (2000,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, labels, test_size=0.3 , random_state = 42)

In [ ]:
_

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [ ]:
preds = classifier.predict(X_test)
preds

array([1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

1.0

## test data

In [ ]:
test_images = load_images("/content/assn2/dummy_submit/test")
cv2_imshow(test_images[2])

7


In [ ]:
test_labels = load_labels("/content/assn2/dummy_submit/test")
test_labels

7


array([0, 0, 0, 1, 1, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, classifier.predict(img_flatten(test_images)))

1.0